In [1]:
import semantic_kernel as sk
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
print(os.getenv("OPENAI_API_BASE"))


https://alkopenai2.openai.azure.com/


In [2]:

from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextCompletion,
)

kernel = sk.Kernel(log=sk.NullLogger())
kernel.add_chat_service(
    "chat_completion",
    AzureChatCompletion(
        "GPT42",
        os.getenv("OPENAI_API_BASE"),
        os.getenv("OPENAI_API_KEY")
    ),
)

In [4]:
# Now we need to import the plugin
from plugins.AudioVideoPlugin.AudioVideo import AudioVideo

# Now you can import the plugin importing skill directly from the function you declared
# in the plugin directory. The import_skill does not need the path, it only need an
# instance of the skill and the name of the skill
extractaudio_plugin = kernel.import_skill(AudioVideo(), skill_name="AudioVideoPlugin")

plugins_directory = "./plugins"

# Import the OrchestratorPlugin from the plugins directory.
publishing_plugin = kernel.import_semantic_skill_from_directory(
    plugins_directory, "PublishingPlugin"
)


In [5]:
from pprint import pprint
# want to print all the keys of extractaudio_plugin that is a dictionary

pprint (extractaudio_plugin["ExtractAudio"])
pprint (extractaudio_plugin["TranscriptTimeline"])
pprint(publishing_plugin["VideoTimelineCreator"])

In [6]:
# Now we can use the skill
# Run the Sqrt function with the context.
result = await kernel.run_async(
    extractaudio_plugin["ExtractAudio"],
    extractaudio_plugin["TranscriptTimeline"],
    input_str="/Users/gianmariaricci/develop/montaggi/UpdatingSSH.mp4",
)

print(result)

/Users/gianmariaricci/develop/github/SemanticKernelPlayground/skernel/lib/python3.11/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


0	9	 Hi and welcome again to a new video about Ubiki in which I deal with a question I got quite often
9	18	 and the question is why I cannot use advanced shared features like SSHK stored in my Ubiki
18	22	 when I work in Windows it seems not supported so what is the problem?
22	29	 This problem usually does not happen in Linux because Linux notively has support for SSH
29	36	 so when you update your Linux machine usually you will receive the latest version of the open SSH protocol
36	40	 that supports Ubiki and you have no problem.
40	45	 In Windows the situation is little different let me show you why.
45	51	 In the vast majority of situations the problem is not having the latest version of SSH
51	59	 and to verify this open a Windows terminal and type SSH, sorry SSH-capital day.
59	66	 And if you did not ever install or upgrade SSH this is the answer.
66	75	 The answer is open SSH for Windows 8.6 and this is indeed too old to support your Ubiki.
75	83	 Now the usual follow-up questi